In [25]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

In [9]:
Datadirectory="training_data/" #training dataset

In [10]:
Classes=["0","1","2","3","4","5","6"]

In [11]:
img_size=224 #to match the imagenet


#### Prepare the Data Generator

In [13]:
def data_generator(batch_size, dataset):
    while True:
        X_batch = []
        y_batch = []
        for img, label in dataset:
            X_batch.append(img)
            y_batch.append(label)
            
            if len(X_batch) == batch_size:
                X_batch = np.array(X_batch) / 255.0  # Normalize
                y_batch = np.array(y_batch)
                yield X_batch, y_batch
                X_batch, y_batch = [], []
        
        if X_batch:
            X_batch = np.array(X_batch) / 255.0
            y_batch = np.array(y_batch)
            yield X_batch, y_batch


#### Load and Split the Data

In [20]:
def create_training_data():
    training_Data = []
    for category in Classes:
        path = os.path.join(Datadirectory, category)
        class_num = Classes.index(category)
        for img in os.listdir(path):
            try:
                img_path = os.path.join(path, img)
                img_array = cv2.imread(img_path)
                new_array = cv2.resize(img_array, (img_size, img_size))
                training_Data.append([new_array, class_num])
            except Exception as e:
                pass
    return training_Data

# Create the dataset
training_Data = create_training_data()

# Split the dataset
train_data, test_data = train_test_split(training_Data, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2


#### Define the CNN Model

In [27]:
model = Sequential([
    Input(shape=(img_size, img_size, 3)),  # Input layer
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(Classes), activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [29]:
batch_size = 32

# Calculate steps per epoch
steps_per_epoch = len(train_data) // batch_size
validation_steps = len(val_data) // batch_size

# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    data_generator(batch_size, train_data),
    steps_per_epoch=steps_per_epoch,
    epochs=20,
    validation_data=data_generator(batch_size, val_data),
    validation_steps=validation_steps,
    callbacks=[early_stopping]
)


Epoch 1/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 594s 1s/step - accuracy: 0.2324 - loss: 1.9672 - val_accuracy: 0.3190 - val_loss: 1.7051
Epoch 2/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 577s 1s/step - accuracy: 0.3080 - loss: 1.7254 - val_accuracy: 0.3599 - val_loss: 1.6411
Epoch 3/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 586s 1s/step - accuracy: 0.3526 - loss: 1.6437 - val_accuracy: 0.4009 - val_loss: 1.5552
Epoch 4/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 14403s 27s/step - accuracy: 0.3924 - loss: 1.5350 - val_accuracy: 0.4142 - val_loss: 1.5026
Epoch 5/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 595s 1s/step - accuracy: 0.4419 - loss: 1.4310 - val_accuracy: 0.4222 - val_loss: 1.5224
Epoch 6/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 575s 1s/step - accuracy: 0.4913 - loss: 1.3119 - val_accuracy: 0.4256 - val_loss: 1.5511
Epoch 7/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 589s 1s/step - accuracy: 0.5382 - loss: 1.1838 - val_accuracy: 0.4433 - val_loss: 1.5790
Epoch 8/20
538/538 ━━━━━━━━━━━━━━━━━━━━ 547s 1s/step - accuracy: 0.5816 - loss: 1.0645 - val_a

In [31]:
# Generate the entire test set
X_test, y_test = [], []
for img, label in test_data:
    X_test.append(img)
    y_test.append(label)

X_test = np.array(X_test) / 255.0
y_test = np.array(y_test)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')


180/180 - 37s - 203ms/step - accuracy: 0.4249 - loss: 1.5047
Test Loss: 1.5046643018722534
Test Accuracy: 0.42493903636932373
